In [1]:
import pymongo
import pandas as pd

In [11]:
#konfigurasi koneksi
username = 'ur_nongodb_username'
password = 'ur_mongodb_password'
host = 'cluster_name.mongodb.net'
dbname = 'ur_database_name'

# URL koneksi MongoDB Atlas
uri = f"mongodb+srv://{username}:{password}@{host}/{dbname}?retryWrites=true&w=majority"

# koneksi ke server MongoDB
client = pymongo.MongoClient(uri)

if client:
    print("Koneksi berhasil terhubung!")

Koneksi berhasil terhubung!


In [3]:
collection_name = 'preprocessing'
db = client[dbname]
collection = db[collection_name]
documents = collection.find()
documents

In [4]:
data = pd.DataFrame(list(documents))
data

,_id,,title,rating,date_of_stay,review,resort_name,preprocessing_title,preprocessing_review
0,6457beaff95018753d35a600,21,A Balinese experience to remember,50,March 2023,We had a wonderful stay in the Joglo Club Vill...,The Legian Seminyak Bali,"['balines', 'experi', 'rememb']","['wonder', 'stay', 'joglo', 'club', 'villa', '..."
1,6457beaff95018753d35a5f2,7,Perfection,50,April 2023,"From the very first moment you arrive, you can...",The Legian Seminyak Bali,['perfect'],"['first', 'moment', 'arriv', 'tell', 'place', ..."
2,6457beaff95018753d35a617,44,Magnificant hotel with wonderful staff,50,January 2023,We spent 3 nights in Villa 4 in the club. We h...,The Legian Seminyak Bali,"['magnific', 'hotel', 'wonder', 'staff']","['spent', '3', 'night', 'villa', '4', 'club', ..."
3,6457beaff95018753d35a5ec,1,Second stay at The Legian in 3 months - fantas...,40,May 2023,It was our second time at The Legian in just 3...,The Legian Seminyak Bali,"['second', 'stay', 'legian', '3', 'month', 'fa...","['second', 'time', 'legian', '3', 'month', 'fb..."
4,6457beaff95018753d35a5f5,10,Perfect in every way…,50,March 2023,We stayed here for five nights and absolutely ...,The Legian Seminyak Bali,"['perfect', 'everi', 'way']","['stay', 'five', 'night', 'absolut', 'love', '..."
...,...,...,...,...,...,...,...,...,...
11323,64583cf7f95018753d35db6a,2366,Random visit but surprisingly very good,40,August 2013,"On our first night at Bali, we randomly visite...",U Paasha Seminyak,"['random', 'visit', 'surprisingli', 'good']","['first', 'night', 'bali', 'randomli', 'visit'..."
11324,64583cf7f95018753d35db70,2372,"A brand new modern hotel, comfortable and central",40,July 2013,We stayed here 3 nights and I have to say It w...,U Paasha Seminyak,"['brand', 'new', 'modern', 'hotel', 'comfort',...","['stay', '3', 'night', 'say', 'perfect', 'plac..."
11325,64583cf7f95018753d35db82,2390,"Smelly, Noisy, 3 rooms in 24hrs!",20,July 2013,We had seen all the great reviews for this pla...,U Paasha Seminyak,"['smelli', 'noisi', '3', 'room', '24hr']","['seen', 'great', 'review', 'place', 'realli',..."
11326,64583cf7f95018753d35db94,2408,Fantastic,50,June 2013,The hotel is great and it is quite new. Locati...,U Paasha Seminyak,['fantast'],"['hotel', 'great', 'quit', 'new', 'locat', 'ba..."


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
vectorizer = CountVectorizer()
# untuk tokenisasi string
analyzer = vectorizer.build_analyzer()

def wordcounts(s):
    c = {}
    # tokenisasi string dan continue, jika tidak empty
    if analyzer(s):
        d = {}
        # mencari data buat di count dan ubah ke array
        w = vectorizer.fit_transform([s]).toarray()
        vc = vectorizer.vocabulary_
        for k,v in vc.items():
            d[v]=k # d -> index:word 
        for index,i in enumerate(w[0]):
            c[d[index]] = i # c -> word:count
    return  c

# menambahkan kolom baru ke dataframe

data['wordcounts_title'] = data['preprocessing_title'].apply(wordcounts)

data.head()

,_id,,title,rating,date_of_stay,review,resort_name,preprocessing_title,preprocessing_review,wordcounts_title
0,6457beaff95018753d35a600,21,A Balinese experience to remember,50,March 2023,We had a wonderful stay in the Joglo Club Vill...,The Legian Seminyak Bali,"['balines', 'experi', 'rememb']","['wonder', 'stay', 'joglo', 'club', 'villa', '...","{'balines': 1, 'experi': 1, 'rememb': 1}"
1,6457beaff95018753d35a5f2,7,Perfection,50,April 2023,"From the very first moment you arrive, you can...",The Legian Seminyak Bali,['perfect'],"['first', 'moment', 'arriv', 'tell', 'place', ...",{'perfect': 1}
2,6457beaff95018753d35a617,44,Magnificant hotel with wonderful staff,50,January 2023,We spent 3 nights in Villa 4 in the club. We h...,The Legian Seminyak Bali,"['magnific', 'hotel', 'wonder', 'staff']","['spent', '3', 'night', 'villa', '4', 'club', ...","{'hotel': 1, 'magnific': 1, 'staff': 1, 'wonde..."
3,6457beaff95018753d35a5ec,1,Second stay at The Legian in 3 months - fantas...,40,May 2023,It was our second time at The Legian in just 3...,The Legian Seminyak Bali,"['second', 'stay', 'legian', '3', 'month', 'fa...","['second', 'time', 'legian', '3', 'month', 'fb...","{'fantast': 1, 'hotel': 1, 'legian': 1, 'month..."
4,6457beaff95018753d35a5f5,10,Perfect in every way…,50,March 2023,We stayed here for five nights and absolutely ...,The Legian Seminyak Bali,"['perfect', 'everi', 'way']","['stay', 'five', 'night', 'absolut', 'love', '...","{'everi': 1, 'perfect': 1, 'way': 1}"


In [8]:
data['wordcounts_review'] = data['preprocessing_review'].apply(wordcounts)

In [9]:
data

,_id,,title,rating,date_of_stay,review,resort_name,preprocessing_title,preprocessing_review,wordcounts_title,wordcounts_review
0,6457beaff95018753d35a600,21,A Balinese experience to remember,50,March 2023,We had a wonderful stay in the Joglo Club Vill...,The Legian Seminyak Bali,"['balines', 'experi', 'rememb']","['wonder', 'stay', 'joglo', 'club', 'villa', '...","{'balines': 1, 'experi': 1, 'rememb': 1}","{'access': 1, 'across': 1, 'also': 2, 'alway':..."
1,6457beaff95018753d35a5f2,7,Perfection,50,April 2023,"From the very first moment you arrive, you can...",The Legian Seminyak Bali,['perfect'],"['first', 'moment', 'arriv', 'tell', 'place', ...",{'perfect': 1},"{'7pm': 1, 'accomod': 1, 'address': 1, 'arriv'..."
2,6457beaff95018753d35a617,44,Magnificant hotel with wonderful staff,50,January 2023,We spent 3 nights in Villa 4 in the club. We h...,The Legian Seminyak Bali,"['magnific', 'hotel', 'wonder', 'staff']","['spent', '3', 'night', 'villa', '4', 'club', ...","{'hotel': 1, 'magnific': 1, 'staff': 1, 'wonde...","{'amaz': 1, 'avail': 1, 'back': 1, 'bit': 1, '..."
3,6457beaff95018753d35a5ec,1,Second stay at The Legian in 3 months - fantas...,40,May 2023,It was our second time at The Legian in just 3...,The Legian Seminyak Bali,"['second', 'stay', 'legian', '3', 'month', 'fa...","['second', 'time', 'legian', '3', 'month', 'fb...","{'fantast': 1, 'hotel': 1, 'legian': 1, 'month...","{'10pm': 1, '7pm': 1, 'account': 1, 'align': 1..."
4,6457beaff95018753d35a5f5,10,Perfect in every way…,50,March 2023,We stayed here for five nights and absolutely ...,The Legian Seminyak Bali,"['perfect', 'everi', 'way']","['stay', 'five', 'night', 'absolut', 'love', '...","{'everi': 1, 'perfect': 1, 'way': 1}","{'absolut': 2, 'attentivehighli': 1, 'butler':..."
...,...,...,...,...,...,...,...,...,...,...,...
11323,64583cf7f95018753d35db6a,2366,Random visit but surprisingly very good,40,August 2013,"On our first night at Bali, we randomly visite...",U Paasha Seminyak,"['random', 'visit', 'surprisingli', 'good']","['first', 'night', 'bali', 'randomli', 'visit'...","{'good': 1, 'random': 1, 'surprisingli': 1, 'v...","{'alibi': 1, 'amaz': 1, 'anyway': 1, 'bali': 1..."
11324,64583cf7f95018753d35db70,2372,"A brand new modern hotel, comfortable and central",40,July 2013,We stayed here 3 nights and I have to say It w...,U Paasha Seminyak,"['brand', 'new', 'modern', 'hotel', 'comfort',...","['stay', '3', 'night', 'say', 'perfect', 'plac...","{'brand': 1, 'central': 1, 'comfort': 1, 'hote...","{'24': 1, '3rd': 1, 'actual': 1, 'air': 1, 'al..."
11325,64583cf7f95018753d35db82,2390,"Smelly, Noisy, 3 rooms in 24hrs!",20,July 2013,We had seen all the great reviews for this pla...,U Paasha Seminyak,"['smelli', 'noisi', '3', 'room', '24hr']","['seen', 'great', 'review', 'place', 'realli',...","{'24hr': 1, 'noisi': 1, 'room': 1, 'smelli': 1}","{'11pm': 1, '15': 1, '150': 1, '233': 1, '24hr..."
11326,64583cf7f95018753d35db94,2408,Fantastic,50,June 2013,The hotel is great and it is quite new. Locati...,U Paasha Seminyak,['fantast'],"['hotel', 'great', 'quit', 'new', 'locat', 'ba...",{'fantast': 1},"{'adult': 1, 'alreadi': 1, 'bad': 1, 'bali': 1..."


In [10]:
filename = 'token_vector_data'
data.to_csv(filename)